<div style="text-align: center;">
<a target="_blank" href="https://colab.research.google.com/github/miquelmn/aa_2526/blob/main/07_Batch_normalization/Batch_normalization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

# Batch Normalization

## Objectius

En aquesta pràctica, ampliarem el treball realitzat amb AlexNet i *transfer learning*, incorporant tècniques de regularització i optimització per millorar el rendiment del model. Els objectius són:

- **Implementar Batch Normalization**: afegir capes de normalització per estabilitzar i accelerar l'entrenament.
- **Comparar resultats**: analitzar l'impacte de cada tècnica en el rendiment final del model.
- **Optimització d'hiperparàmetres**: provar diferents configuracions per trobar la millor combinació.

Aquest enfocament permetrà comprendre com les tècniques vistes a teoria milloren la generalització i eviten l'overfitting en problemes reals de classificació d'imatges.

Una segona part serà emprar els nous models vists a classe de teoria.

## Introducció

### Batch Normalization

La Batch Normalization és una tècnica que normalitza les activacions de cada capa durant l'entrenament, utilitzant la mitjana i la desviació estàndard del mini-batch actual. Els seus principals avantatges són:

- **Accelera l'entrenament**: permet utilitzar learning rates més alts.
- **Redueix la sensibilitat a la inicialització**: els pesos inicials tenen menys impacte.
- **Actua com a regularitzador**: redueix la necessitat de Dropout en alguns casos.
- **Millora la convergència**: facilita que el model arribi a millors mínims.


La formulació d'aquesta operació, tal com heu vist a classe de teoria, és la següent:

$$ \hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$

$$ y_i = \gamma \hat{x}_i + \beta, $$

on $\gamma$ i $\beta$ són paràmetres entrenables.

Per implementar-ho feim operacions diferents a entrenament i validació.

In [ ]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import torch.optim as optim
from torchvision import datasets, models, transforms

import numpy as np
from sklearn.model_selection import train_test_split

class MyBatchNorm1d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.num_features = num_features

        # Evitar divisió per 0
        self.eps = eps

        # Momentum
        self.momentum = momentum

        # Paràmetres aprenables: gamma (weight) i beta (bias)
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

        # Estadístiques que s’acumulen durant l’entrenament però que no són paràmetres
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def forward(self, x):
        if self.training:
            # Calcular mitjana i variància del batch
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)

            # Actualitzar estadístiques globals
            self.running_mean = self.momentum * batch_mean + (1 - self.momentum) * self.running_mean
            self.running_var = self.momentum * batch_var + (1 - self.momentum) * self.running_var

            # Normalitzar el batch actual
            x_hat = (x - batch_mean) / (torch.sqrt(batch_var + self.eps))
        else:
            # En mode d’avaluació, s’usen les estadístiques acumulades
            x_hat = (x - self.running_mean) / (torch.sqrt(self.running_var+ self.eps))

        # Aplicar gamma i beta
        y = self.gamma * x_hat + self.beta
        return y


In [32]:
import random

BATCH_SIZE = 4
EPOCHS = 5
SAMPLE = 50000

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(10)
])

whole_dataset = datasets.ImageFolder('../tiny-imagenet-200/train', transform=transform)
idx_datasets = np.arange(len(whole_dataset))

# Subsampling

subset = random.choices(idx_datasets, k=SAMPLE)
train, test = train_test_split(subset)



train = torch.utils.data.Subset(whole_dataset, train)
test = torch.utils.data.Subset(whole_dataset, test)

# test = datasets.ImageFolder('../../data/tiny-imagenet-200/test', transform=transform)

train_loader = torch.utils.data.DataLoader(train,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

In [33]:
alex = models.alexnet(weights=True)
# Freezing the feature extractor
for param in alex.features.parameters():
  param.requires_grad = False


c:\Users\Marc\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
alex.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    MyBatchNorm1d(num_features=1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 200),  # Ja que tenim 200 classes.
)  # Ja que és multiclasse.


In [35]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3  # Hiperparàmetre
optimizer = optim.Adam(alex.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = alex.to(device)

In [36]:
from sklearn.metrics import accuracy_score

running_loss = []
running_acc = []

running_test_loss = []
running_test_acc_cnn = []

for t in tqdm(range(EPOCHS), desc="Èpoques"):
    batch_loss = 0
    batch_acc = 0

    i_batch = 1
    # Iteram els batches.
    for i_batch, (x, y) in tqdm(enumerate(train_loader), desc=f"Batches (Època {t + 1})", leave=False):
        alex.train()  # Posam el model a mode entranament.

        optimizer.zero_grad()

        # 1. PREDICCIÓ
        y_pred = alex(x.to(device))

        # 2. CALCUL DE LA PÈRDUA
        # Computa la pèrdua: l'error de predicció vs el valor correcte
        # Es guarda la pèrdua en un array per futures visualitzacions

        loss = loss_fn(y_pred, y.to(device))

        #3. GRADIENT
        alex.zero_grad()
        loss.backward()

        # Actualitza els pesos utilitzant l'algorisme d'actualització
        #4. OPTIMITZACIO
        with torch.no_grad():
            optimizer.step()

        # 5. EVALUAM EL MODEL
        alex.eval()  # Mode avaluació de la xarxa

        y_pred = alex(x.to(device))
        batch_loss += (loss_fn(y_pred, y.to(device)).detach())

        y_pred_class = torch.argmax(y_pred, dim=1)
        batch_acc += accuracy_score(y, y_pred_class.detach().cpu().numpy())

    running_loss.append(batch_loss / (i_batch + 1))
    running_acc.append(batch_acc / (i_batch + 1))

    batch_test_loss = 0
    batch_test_acc = 0

    alex.eval()
    for i_batch, (x, y) in enumerate(test_loader):
        y_pred = alex(x.to(device))
        batch_test_loss += (loss_fn(y_pred, y.to(device)).detach())

        y_pred_class = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
        batch_test_acc += accuracy_score(y, y_pred_class)

    running_test_loss.append(batch_test_loss / (i_batch + 1))
    running_test_acc_cnn.append(batch_test_acc / (i_batch + 1))
    print(f"Època {t + 1} finalitzada. "
          f"Pèrdua entrenament: {running_loss[-1]:.4f}, "
          f"Precisió entrenament: {running_acc[-1]:.4f}, "
          f"Pèrdua test: {running_test_loss[-1]:.4f}, "
          f"Precisió test: {running_test_acc_cnn[-1]:.4f}")

Èpoques:   0%|          | 0/5 [11:55<?, ?it/s]


KeyboardInterrupt: 